<a href="https://colab.research.google.com/github/huuphong1995/hello-world/blob/master/Test_BGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pickle
with open('BGL-train.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
x_tr = data['x_tr']
y_tr = data['y_tr']

In [4]:
# Separate the anomaly and normal instances in the training set
anomalies_tr = [x for x, y in zip(x_tr, y_tr) if y == 1]
normal_tr = [x for x, y in zip(x_tr, y_tr) if y == 0]

In [5]:
import random
# Randomly sample equal number of anomalies and normal instances
num_samples_tr = min(len(anomalies_tr), len(normal_tr))
sampled_anomalies_tr = random.sample(anomalies_tr, num_samples_tr)
sampled_normal_tr = random.sample(normal_tr, num_samples_tr)

In [6]:
# Combine the sampled anomalies and normal instances for the balanced training set
x_tr_balanced = sampled_anomalies_tr + sampled_normal_tr
y_tr_balanced = [1] * num_samples_tr + [0] * num_samples_tr

In [7]:
from sklearn.utils import shuffle
x_tr_balanced, y_tr_balanced = shuffle(x_tr_balanced, y_tr_balanced)

In [8]:
len(x_tr_balanced) - len(y_tr_balanced)

0

In [12]:
import numpy as np
x_tr_balanced = np.array(x_tr_balanced)
y_tr_balanced = np.array(y_tr_balanced)

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
# Define the CNN-based model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(20, 768)))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_tr_balanced, y_tr_balanced, batch_size=32, epochs=10)


Epoch 1/10
1079/1079 [==============================] - 25s 22ms/step - loss: 0.0702 - accuracy: 0.9777
Epoch 2/10
1079/1079 [==============================] - 23s 22ms/step - loss: 0.0268 - accuracy: 0.9933
Epoch 3/10
1079/1079 [==============================] - 24s 22ms/step - loss: 0.0219 - accuracy: 0.9949
Epoch 4/10
1079/1079 [==============================] - 22s 20ms/step - loss: 0.0208 - accuracy: 0.9954
Epoch 5/10
1079/1079 [==============================] - 22s 20ms/step - loss: 0.0177 - accuracy: 0.9964
Epoch 6/10
1079/1079 [==============================] - 21s 19ms/step - loss: 0.0180 - accuracy: 0.9963
Epoch 7/10
1079/1079 [==============================] - 22s 20ms/step - loss: 0.0171 - accuracy: 0.9970
Epoch 8/10
1079/1079 [==============================] - 22s 20ms/step - loss: 0.0173 - accuracy: 0.9966
Epoch 9/10
1079/1079 [==============================] - 21s 19ms/step - loss: 0.0167 - accuracy: 0.9968
Epoch 10/10
1079/1079 [==============================] - 22s 20m

###Test the model with the testing set


In [14]:

with open('BGL-test.pkl', 'rb') as f:
    data1 = pickle.load(f)

In [15]:
x_te = data1['x_te']
y_te = data1['y_te']

In [16]:
# Separate the anomaly and normal instances in the testing set
anomalies_te = [x for x, y in zip(x_te, y_te) if y == 1]
normal_te = [x for x, y in zip(x_te, y_te) if y == 0]

# Randomly sample equal number of anomalies and normal instances for testing
num_samples_te = min(len(anomalies_te), len(normal_te))
sampled_anomalies_te = random.sample(anomalies_te, num_samples_te)
sampled_normal_te = random.sample(normal_te, num_samples_te)

# Combine the sampled anomalies and normal instances for the balanced testing set
x_te_balanced = sampled_anomalies_te + sampled_normal_te
y_te_balanced = [1] * num_samples_te + [0] * num_samples_te

In [17]:
x_te_balanced, y_te_balanced = shuffle(x_te_balanced, y_te_balanced)

In [18]:
x_te_balanced = np.array(x_te_balanced)
y_te_balanced = np.array(y_te_balanced)

In [19]:
# Evaluate the model
loss, accuracy = model.evaluate(x_te_balanced, y_te_balanced)
print('Accuracy:', accuracy)

188/188 [==============================] - 2s 8ms/step - loss: 0.0947 - accuracy: 0.9795
Accuracy: 0.9795272946357727


In [47]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score

# Make predictions on the balanced testing set
y_pred_prob = model.predict(x_te_balanced)
y_pred = np.round(y_pred_prob).astype(int)

# Calculate the F1-score
f1 = f1_score(y_te_balanced, y_pred)
print('F1-score:', f1)

# Calculate the recall
recall = recall_score(y_te_balanced, y_pred)
print('Recall:', recall)


188/188 [==============================] - 1s 8ms/step
F1-score: 0.9791419365779209
Recall: 0.961051930758988


In [53]:
from sklearn.metrics import precision_score
# Calculate the precision
precision = precision_score(y_te_balanced, y_pred)
print('Precision:', precision)

Precision: 0.9979260283442793


In [48]:
len([x for x in y_te_balanced if x==0])

3004

In [49]:
# Calculate Recall and F1-score
def calculate_metrics(y_true, y_pred):
    TP = sum((y_true == 1) & (y_pred == 1))
    FP = sum((y_true == 0) & (y_pred == 1))
    FN = sum((y_true == 1) & (y_pred == 0))

    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return recall, f1_score

In [50]:
type(y_pred)

numpy.ndarray

In [51]:
type(y_pred[1])

numpy.ndarray